# Implementering av RVT
Här används en implementering av RVT gjord av SandoghdarLab som kan hittas här: https://github.com/SandoghdarLab/rvt


In [ ]:
# import dependencies
import imgrvt as rvt
import numpy as np
import matplotlib.pyplot as plt
import os
import deeptrack as dt
import cv2
import torch
import matplotlib.pyplot as plt
import torch
import deeplay as dl
from torch.utils.data import DataLoader
from torch.optim import Adam
import matplotlib.pyplot as plt
import datetime
import os
import numpy as np
import deeptrack as dt
from PIL import Image
from torchvision import transforms
from skimage import morphology
import pandas as pd
import h5py

### Läs av data från .mat filer 
Datan är given i .mat filer som här görs om till ett dataset

In [ ]:
# file path for the .mat file
file_path = os.getcwd() + '/Polystyrene_dia100nm_gap500nm_bgSubtracted.mat'

f = h5py.File(file_path, 'r')
data = f['Im_stack']

# show the first image in the data
plt.imshow(data[0], cmap='gray')


### Applicera RVT på en bild
Här appliceras RVT på en bild för att visa detektioner i en bildruta. Först används ett gaussiskt filter för att ta bort högfrekvent brus i datan. Därefter transformeras bilden med RVT och partikelpositioner tas ut av att hitta lokala maximan i bilden.

In [ ]:
from skimage.filters import gaussian
# apply gaussian filter to the image
frame_gauss = gaussian(data[0], sigma=1)

# parameters for the RVT
rmin = 1
rmax = 10
threshold = 0.99

# apply the RVT
th = np.quantile(rvt.rvt(frame_gauss,rmin=rmin,rmax=rmax), threshold)
timg1 = rvt.rvt(frame_gauss,rmin=1,rmax=10, pad_mode='constant')

# find the coordinates of the detections
coordinates = morphology.h_maxima(np.squeeze(timg1), th) == 1
coordinates = np.where(coordinates == True)

#plot the detections on one frame
plt.imshow(data[0].squeeze(), cmap='gray')
plt.axis('off')
plt.scatter(coordinates[1], coordinates[0], label='Predicted', s=100, facecolor='none', linewidths=1, edgecolors='r')
plt.savefig('RVT500pred.pdf', bbox_inches='tight')

### Detektera för alla bilder i datasetet

In [ ]:
from tqdm import tqdm

# initialize dataframe
df = pd.DataFrame(columns=['centroid-0', 'centroid-1', 'frame', 'label', 'set', 'solution'])
df1 = pd.DataFrame(columns=['centroid-0', 'centroid-1', 'frame', 'label', 'set', 'solution'])

# initialize counter
i = 0

# parameters for the RVT
rmin = 1
rmax = 10
threshold = 0.99

for frame in tqdm(data):

    # apply gaussian filter to the image
    gauss_frame = gaussian(frame, sigma=1)
    
    # apply the RVT
    th = np.quantile(rvt.rvt(gauss_frame,rmin=rmin,rmax=rmax), threshold)
    timg1 = rvt.rvt(gauss_frame,rmin=1,rmax=10, pad_mode='constant')
    
    # find the coordinates of the detections
    coordinates = morphology.h_maxima(np.squeeze(timg1), th) == 1
    coordinates = np.where(coordinates == True)

    # add the coordinates to the dataframe
    df1 = pd.DataFrame(columns=['centroid-0', 'centroid-1', 'frame', 'label', 'set', 'solution'])
    df1['centroid-0'] = coordinates[0]/frame.shape[0]
    df1['centroid-1'] = coordinates[1]/frame.shape[1]
    df1['frame'] = int(i)
    df1['label'] = int(0)
    df1['set'] = int(0)
    df1['solution'] = int(0)
    df = pd.concat([df, df1], ignore_index=True)
    i += 1
    
    # reset the df1 dataframe for next iteration
    df1.loc[0:0]

df

### Spara datan till en csv fil för att användas med MAGIK

In [ ]:
file_name = 'RVTpredictions'

df.to_csv(file_name + '.csv', index=False)